# CSV_Converter

## This notebook converts an imagery dataset to CSV files


In [1]:
import coast_corr
import warnings
#warnings.filterwarnings("ignore")
import osmnx as ox
from tkinter import filedialog
from tkinter import *
import ipywidgets
import os
import glob
import pandas as pd
import rasterio as rio
import rioxarray
import geopandas as gpd
import tqdm

In [2]:
root = Tk()
root.withdraw()
outfolder = filedialog.askdirectory()

band=ipywidgets.Text(
    value='1',
    placeholder='Type something',
    description='Band:',
    disabled=False
)
display(band)

Text(value='1', description='Band:', placeholder='Type something')

In [13]:
inputs=outfolder

#for function:

#First Create Folder for CSV's in parent folder of directory
path_parent = os.path.dirname(inputs)
new_fold=path_parent+'/csv'
if not os.path.exists(new_fold): os.makedirs(new_fold)
csv_bandfld=new_fold+"/"+"Band"+band.value
if not os.path.exists(csv_bandfld): os.makedirs(csv_bandfld)
csv_out=csv_bandfld
#make a df of all images in folder
clipping_df=pd.DataFrame(columns=["Rrs_files","date","time"])
clipping_df['Rrs_files']=glob.glob(inputs+'/*.tif')
print("Converting images to csv's, please wait...", end='\r')

for i in range(0,len(clipping_df)):
    date, time = clipping_df['Rrs_files'][i].split('/')[len(clipping_df['Rrs_files'][i].split('/'))-1].split('_')[0][:10],clipping_df['Rrs_files'][i].split('/')[len(clipping_df['Rrs_files'][i].split('/'))-1].split('_')[0][11:]
    clipping_df['date'][i]=date
    clipping_df['time'][i]=time
    rds = rioxarray.open_rasterio(clipping_df['Rrs_files'][i])
    rds.name = "data"
    df = rds.squeeze().to_dataframe().reset_index()
    df=df[df['band']==int(band.value)]
    gdf = gpd.GeoDataFrame(df, crs=rds.rio.crs, geometry=gpd.points_from_xy(df.x, df.y))
    #gdf['crs']=str(rds.rio.crs)
    gdf['Date']=clipping_df['date'][i]
    gdf['Time']=clipping_df['time'][i]
    date_name=clipping_df['date'][i]
    csv_filename=csv_out+"/"+date_name+'.csv'
    gdf.to_csv(csv_filename)
    i=i+1
    frac=round(((i/len(clipping_df))*100),2)
    print("Converting images to csv's, please wait...", frac,"%", end='\r')

print("Done!")


Done!rting images to csv's, please wait... 100.0 %
